# 데이터 실수화

In [68]:
# 범주형 데이터 실수화와 범주형 변수의 연속형 변수 변환
# 1. 범주형 데이터의 실수화
# 1) 필요 라이브러리 import
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [69]:
# 2) 데이터 set 생성 및 확인

# 데이터 set 생성
x_train = pd.DataFrame(["남성", "여성", "남성", "여성", "남성", "여성"], columns=["성별"])

# 데이터 확인
x_train.head(3)

,성별
0,남성
1,여성
2,남성


In [71]:
# 데이터 프레임의 정보 확인 : object 타입 '성별' 변수
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [72]:
# 범주형 데이터의 갯수 확인
x_train['성별'].value_counts()

여성    3
남성    3
Name: 성별, dtype: int64

In [73]:
# 3) 레이블 인코딩 : 문자를 0부터 시작하는 정수형 숫자로 변환

# 레이블 인코더 생성
encoing = LabelEncoder()
# X_train 데이터를 이용하여 피팅하고 레이블 숫자로 변환
encoing.fit(x_train["성별"])
x_train["성별_인코딩"] = encoing.transform(x_train['성별'])

In [74]:
# 4) 인코딩 결과 확인
x_train

,성별,성별_인코딩
0,남성,0
1,여성,1
2,남성,0
3,여성,1
4,남성,0
5,여성,1


In [19]:
# 클래스 확인
print(list(encoing.classes_))
# 인코딩 값으로 문자값 확인
print(list(encoing.inverse_transform([1,0])))

['남성', '여성']
['여성', '남성']


In [75]:
# 2. One-Hot Encoding : 더미변수 생성, 가변환

# 1) 데이터 확인(실수화한 데이터 사용)
x_train.head(2)

,성별,성별_인코딩
0,남성,0
1,여성,1


In [76]:
# 2) One-Hot Encoding

# 원핫인코더 생성
# sparse를 True로 할 경우 "(행, 열) 1"의 좌표리스트의 형식, False로 할 경우 넘파이 배열로 변환
one_encoding = OneHotEncoder(sparse=False)
# X_train 데이터를 이용하여 피팅
one_encoding.fit(x_train[["성별"]])
# 가변환값 변환
one_encoding.transform(x_train[["성별"]])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [22]:
# 가변환 된 피쳐 확인
print(one_encoding.get_feature_names())

['x0_남성' 'x0_여성']


In [77]:
# 3) 인코딩 결과 확인

# 가변환된 값을 x_train_one 데이터 프레임으로 저장
x_train_one = pd.DataFrame(one_encoding.transform(x_train[['성별']]), columns=['성별0','성별1'])

In [78]:
# x_train 데이터와 x_train_one의 가변환된 속성 합침
x_train=pd.concat([x_train, x_train_one], axis=1)

In [32]:
# 결과 확인
x_train.head(3)

,성별,성별_인코딩,성별0,성별1
0,남성,0,1.0,0.0
1,여성,1,0.0,1.0
2,남성,0,1.0,0.0


## 텍스트 데이터 실수화
### 1. 단어의 빈도수를 카운트(Count)하여 단어를 수치화하는 방법
### -CountVectorizer 함수 : 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 Bag Of Words로 인코딩하는 함수

In [81]:
# 필요 라이브러리 import
from sklearn.feature_extraction.text import CountVectorizer
corpus  = ['청년 인재 개발 양성 과정','인공지능 청년 양성','미래 인공지능 데이터 대한민국','데이터 원유 기술사 청년 개발']

In [84]:
# 데이터셋 생성 및 확인
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [83]:
type(corpus)

list

In [85]:
# 텍스트 실수화 : 단어의 빈도수를 카운트(Count)
count_vect = CountVectorizer()
count_vect.fit(corpus)
count_vect.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [86]:
# 단어 벡터화한 값을 array로 변환하여 확인
count_vect.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [87]:
# 단어 벡터화
features = count_vect.transform(corpus)

In [88]:
features

<4x11 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [90]:
# 4) 문서단어행렬로 변환

# 속성 이름만 반환
vocab = count_vect.get_feature_names()

In [91]:
vocab

['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [94]:
# 문서단어행렬(DTM)을 데이터 프레임으로 변환
DTM = pd.DataFrame(features.toarray(), columns = vocab).head()

In [97]:
DTM

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,1,1,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,1,0,1,0,1
2,0,0,0,1,1,1,0,0,1,0,0
3,1,0,1,0,1,0,0,1,0,0,1


In [99]:
# 문서단어행렬에 일치하는 단어 확인
count_vect.transform(['기술사 대한민국 인재 만세']).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

# 텍스트 데이터 실수화 실습
## 2. 단어의 빈도수를 단어들마다 중요한 정도를 가중치로 주어 수치화하는 방법
## TfidfVectorizer 함수 : TF-IDF 방식으로 단어의 가중치를 조정한 BOW로 인코딩 하는 함수


In [100]:
# 1) 필요 라이브러리 import
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
# 2) 데이터 set 생성 및 확인
# -단어의 빈도수 수치화에서 활용한 corpus 데이터셋 사용
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [102]:
# 3) 텍스트 실수화 : TF-IDF 방식

# TF-IDF 벡터라이저 생성
tfid = TfidfVectorizer()
# 단어 카운트
tfid.fit(corpus)
# 단어사전 확인
tfid.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [103]:
# 단어 벡터화한 값을 array로 변환하여 확인
tfid.transform(corpus).toarray()

array([[0.41263976, 0.52338122, 0.        , 0.        , 0.        ,
        0.        , 0.41263976, 0.        , 0.        , 0.52338122,
        0.33406745],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.61366674, 0.        , 0.61366674, 0.        ,
        0.49681612],
       [0.        , 0.        , 0.        , 0.55528266, 0.43779123,
        0.55528266, 0.        , 0.        , 0.43779123, 0.        ,
        0.        ],
       [0.41263976, 0.        , 0.52338122, 0.        , 0.41263976,
        0.        , 0.        , 0.52338122, 0.        , 0.        ,
        0.33406745]])

In [104]:
# 단어 벡터화
features_idf = tfid.transform(corpus)

In [105]:
features_idf

<4x11 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [110]:
# 4) 문서단어행렬로 변환

# 속성 이름만 반환
vocab_idf = count_vect.get_feature_names()

In [111]:
vocab_idf

['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [112]:
# 문서단어행렬(DTM)을 데이터 프레임으로 변환
DTM_idf = pd.DataFrame(features_idf.toarray(), columns = vocab_idf).head()

In [113]:
DTM_idf

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,0.41264,0.523381,0.000000,0.000000,0.000000,0.000000,0.412640,0.000000,0.000000,0.523381,0.334067
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.613667,0.000000,0.613667,0.000000,0.496816
2,0.00000,0.000000,0.000000,0.555283,0.437791,0.555283,0.000000,0.000000,0.437791,0.000000,0.000000
3,0.41264,0.000000,0.523381,0.000000,0.412640,0.000000,0.000000,0.523381,0.000000,0.000000,0.334067
